In [1]:
from pathlib import Path
import os

yr = 2023
d = 5

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
def format_input(inp):
  lines = inp.split('\n')
  seed_nums = [int(n) for n in lines[0].split(' ') if n.isnumeric()]


  output = [seed_nums]
  cur_dict = None
  for l in lines[1:]:
    if '-' in l:
      if cur_dict is not None:
        output.append(cur_dict)
      cur_dict = {}
    if len(l.split(' ')) != 0 and l.split(' ')[0].isnumeric():
      cur_nums = [int(n) for n in l.split(' ')]
      cur_dict[cur_nums[1]] = {}
      cur_dict[cur_nums[1]]['dest'] = cur_nums[0]
      cur_dict[cur_nums[1]]['range'] = cur_nums[2]
  output.append(cur_dict)

  return tuple(output)




In [3]:
def get_value(q, d):
  for k, v in d.items():
    if q >= k and q <= k+v['range']-1:
      return q-k+v['dest']
  else:
    return q

def get_value_chained(q, *args):
  for d in args:
    q = get_value(q, d)
  return q

def get_seed_locs(seed_nums, *args):
  seed_locs = []
  for sv in seed_nums:
    seed_locs.append(get_value_chained(sv, *args))
  return seed_locs

In [4]:
def get_potential_values(q, qr, d):
  qmax = q+qr-1
  potential_dests = []
  found_ranges = []
  for k, v in d.items():
    # If our query overlaps with the current range
    if qmax >= k and q <= k+v['range']-1:
      min_dest = max(v['dest'], v['dest']+(q-k))
      max_dest = min(v['dest']+v['range']-1,
                     v['dest']+v['range']-1-(k+v['range']-1-qmax))
      min_q_overlap = max(k, q)
      max_q_overlap = min(k+v['range']-1, qmax)

      potential_dests.append((min_dest, max_dest))
      found_ranges.append((min_q_overlap, max_q_overlap))

  found_ranges = sorted(found_ranges, key=lambda x: x[0])


  # For all ranges in our query that we didn't find a match for in our
  # dictionary, map those ranges to themselves in our potential destinations
  unfound_ranges = []
  for i, fr in enumerate(found_ranges):
    if i==len(found_ranges)-1:
      if fr[1] != qmax:
        unfound_ranges.append((fr[1]+1, qmax))
    elif fr[1] != found_ranges[i+1][0]:
      unfound_ranges.append((fr[1]+1, found_ranges[i+1][0]-1))
    if i==0:
      if fr[0] != q:
        unfound_ranges.append((q, fr[0]-1))
  if len(found_ranges)==0:
    unfound_ranges = [(q,qmax)]


  unfound_ranges = [ur for ur in unfound_ranges if ur[0]<=ur[1]]
  for ur in unfound_ranges:
    potential_dests.append(ur)


  # Convert Destinations to be destinations with ranges
  potential_dests = [(pd[0], pd[1]-pd[0]+1) for pd in potential_dests]

  return sorted(potential_dests, key=lambda x: x[0])


def get_potential_values_chained(q, qr, *args):
  potential_vals = [(q,qr)]
  next_potential_vals = []

  for d in args:
    for pv in potential_vals:
      cur_pvs = get_potential_values(pv[0], pv[1], d)
      for cpv in cur_pvs:
        next_potential_vals.append(cpv)
    potential_vals = sorted(next_potential_vals, key=lambda x: x[0])
    next_potential_vals = []
  return potential_vals


def get_seed_range_locs(seed_nums, *args):
  seed_ranges = [(seed_nums[i], seed_nums[i+1])
                   for i in range(0,len(seed_nums),2)]
  seed_locs = []
  for sr in seed_ranges:
    cur_seed_locs = get_potential_values_chained(sr[0], sr[1], *args)
    for csl in cur_seed_locs:
      seed_locs.append(csl)
  return seed_locs

In [5]:
import time

t = time.time()

(seed_nums, seed_2_soil, soil_2_fertilizer, fertilier_2_water, water_2_light,
light_2_temperature, temperature_2_humidity,
humidity_2_location) = format_input(inp)

print(min(get_seed_locs(seed_nums,
              seed_2_soil,
              soil_2_fertilizer,
              fertilier_2_water,
              water_2_light,
              light_2_temperature,
              temperature_2_humidity,
              humidity_2_location)))

print(min(get_seed_range_locs(seed_nums,
              seed_2_soil,
              soil_2_fertilizer,
              fertilier_2_water,
              water_2_light,
              light_2_temperature,
              temperature_2_humidity,
              humidity_2_location), key=lambda x: x[0])[0])

print("\nRUNTIME: ", time.time()-t)

662197086
52510809

RUNTIME:  0.0029914379119873047
